## Ionic Emission Analysis


In [1]:
pip install dune-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.1 MB/s eta 0:00:00


## To refresh data, select Runtime > Run All and allow permissions when prompted.




In [2]:
from dune_client.client import DuneClient
from google.colab import userdata
userdata.get('DUNE_API_KEY')



dune_api_key = userdata.get('DUNE_API_KEY')
dune = DuneClient(dune_api_key)
deposits = dune.get_latest_result_dataframe(4003406) # mrwild
print("deposits loaded")
borrows= dune.get_latest_result_dataframe(4301345) # mrwild
print("borrows loaded")
withdrawals = dune.get_latest_result_dataframe(4003238) # mrwild
print("withdrawals loaded")
tvl_agg = dune.get_latest_result_dataframe(4001052) # mrwild TVL by POOL
print("tvl by pool loaded")
protocol_tvl = dune.get_latest_result_dataframe(4301363) # NEW  TVL cumulative (old 4069195)
print("total tvl loaded")
protocol_tvl_notional = dune.get_latest_result_dataframe(4309385) # NEW TVL cumulative value (UZL)
print("protocol_tvl_notional loaded")


deposits loaded
borrows loaded
withdrawals loaded
tvl by pool loaded
total tvl loaded
protocol_tvl_notional loaded


In [3]:
# copies of the dataframes
deposits_copy = deposits.copy()
borrows_copy = borrows.copy()
withdrawals_copy = withdrawals.copy()
tvl_agg_copy = tvl_agg.copy()
protocol_tvl_copy = protocol_tvl.copy()
protocol_tvl_notional_copy = protocol_tvl_notional.copy()


In [4]:
deposits_copy.tail(5)

,date,vaultName,mintedAmount_USD
4813,2024-12-04 00:00:00.000 UTC,ionsUSDem,668.568189
4814,2024-12-04 00:00:00.000 UTC,ionwUSDMb,100.367283
4815,2024-12-04 00:00:00.000 UTC,ionweETHm,5.855677
4816,2024-12-04 00:00:00.000 UTC,ionwrsETHm,5.399112
4817,2024-12-04 00:00:00.000 UTC,ionwsuperOETHb,0.093990


In [5]:
borrows_copy.tail(5)

,date,vaultName,daily_borrowed_amount_usd,total_borrowed_in_vault_usd
16606,2024-01-01,ionweETHm,0.0,<nil>
16607,2024-01-01,ionwrsETHm,0.0,<nil>
16608,2024-01-01,ionwstETHb,0.0,<nil>
16609,2024-01-01,ionwstETHo,0.0,<nil>
16610,2024-01-01,ionwsuperOETHb,0.0,<nil>


In [6]:
withdrawals_copy.tail(5)

,date,vaultName,redeemedAmount_USD
4875,2024-12-04 00:00:00.000 UTC,ionsUSDzb,300.044497
4876,2024-12-04 00:00:00.000 UTC,ionweETHb,16.403964
4877,2024-12-04 00:00:00.000 UTC,ionweETHm,6.235062
4878,2024-12-04 00:00:00.000 UTC,ionwrsETHm,0.007894
4879,2024-12-04 00:00:00.000 UTC,ionwsuperOETHb,0.093990


In [7]:
tvl_agg_copy.tail(5)

,vaultName,TVL_USD,TotalBorrowed_USD,ActiveDeposits_USD
44,ioneUSDb,179822.242772,8.655264e+05,1.045349e+06
45,ionAEROb,139346.406401,1.491636e+05,2.885100e+05
46,ionbsdETH,339867.471150,3.850409e+05,7.249084e+05
47,ionWETHb,650829.470357,1.147480e+06,1.798309e+06
48,ionUSDCb,537188.112669,1.289691e+06,1.826879e+06


In [8]:
protocol_tvl_copy.tail(5)

,date,ionicVault,TVL,chain
16606,2024-12-04,ionweETHm,7.702361e+05,mode
16607,2024-12-04,ionwrsETHm,9.933368e+05,mode
16608,2024-12-04,ionwstETHb,1.242850e+05,base
16609,2024-12-04,ionwstETHo,2.462299e+01,optimism
16610,2024-12-04,ionwsuperOETHb,1.288179e+06,base


In [9]:
protocol_tvl_notional_copy.tail(5)

,date,ionicVault,TVL,chain
16606,2024-12-04,ionweETHm,202.478087,mode
16607,2024-12-04,ionwrsETHm,261.126329,mode
16608,2024-12-04,ionwstETHb,29.031249,base
16609,2024-12-04,ionwstETHo,0.005752,optimism
16610,2024-12-04,ionwsuperOETHb,356.945528,base


## Current Epoch Emissions

In [10]:
import pandas as pd
import numpy as np
# Convert date columns to datetime with consistent timezones for copied dataframes
withdrawals_copy['date'] = pd.to_datetime(withdrawals_copy['date'], utc=True)
deposits_copy['date'] = pd.to_datetime(deposits_copy['date'], utc=True)
borrows_copy['date'] = pd.to_datetime(borrows_copy['date'], utc=True)
protocol_tvl_copy['date'] = pd.to_datetime(protocol_tvl_copy['date'], utc=True)
protocol_tvl_notional_copy['date'] = pd.to_datetime(protocol_tvl_notional_copy['date'], utc=True)

# Rename columns in copied dataframes
protocol_tvl_copy = protocol_tvl_copy.rename(columns={'ionicVault': 'vaultName'})
protocol_tvl_notional_copy = protocol_tvl_notional_copy.rename(columns={'ionicVault': 'vaultName', 'TVL': 'TVL_notional'})

###########################
# INPUT START AND END DATES
###########################
start_date = pd.Timestamp('2024-11-01', tz='UTC')
end_date = pd.Timestamp('2024-12-04', tz='UTC')
###########################
# INPUT START AND END DATES
###########################


# TVL at Start and End (USD and Asset)
tvl_start_usd = protocol_tvl_copy[protocol_tvl_copy['date'] == start_date][['vaultName', 'TVL']]
tvl_start_usd = tvl_start_usd.rename(columns={'TVL': 'TVL at Start (USD)'})
tvl_end_usd = protocol_tvl_copy[protocol_tvl_copy['date'] == end_date][['vaultName', 'TVL']]
tvl_end_usd = tvl_end_usd.rename(columns={'TVL': 'TVL at End (USD)'})

tvl_start_asset = protocol_tvl_notional_copy[protocol_tvl_notional_copy['date'] == start_date][['vaultName', 'TVL_notional']]
tvl_start_asset = tvl_start_asset.rename(columns={'TVL_notional': 'TVL at Start (Asset)'})
tvl_end_asset = protocol_tvl_notional_copy[protocol_tvl_notional_copy['date'] == end_date][['vaultName', 'TVL_notional']]
tvl_end_asset = tvl_end_asset.rename(columns={'TVL_notional': 'TVL at End (Asset)'})

tvl = pd.merge(tvl_start_usd, tvl_end_usd, on='vaultName', how='outer')
tvl = pd.merge(tvl, tvl_start_asset, on='vaultName', how='outer')
tvl = pd.merge(tvl, tvl_end_asset, on='vaultName', how='outer')

# Total Borrows at Start and End
def get_latest_borrowed_amount(borrows_df, date_limit):
    borrows_filtered = borrows_df[borrows_df['date'] <= date_limit]
    borrows_sorted = borrows_filtered.sort_values(by=['vaultName', 'date'])
    latest_borrows = borrows_sorted.groupby('vaultName').last().reset_index()
    return latest_borrows[['vaultName', 'total_borrowed_in_vault_usd']]

borrows_start = get_latest_borrowed_amount(borrows_copy, start_date)
borrows_start = borrows_start.rename(columns={'total_borrowed_in_vault_usd': 'Total Borrows at Start (USD)'})
borrows_end = get_latest_borrowed_amount(borrows_copy, end_date)
borrows_end = borrows_end.rename(columns={'total_borrowed_in_vault_usd': 'Total Borrows at End (USD)'})
borrows_merged = pd.merge(borrows_start, borrows_end, on='vaultName', how='outer').fillna(0)


# mergeTVL and Borrows data
final_df = pd.merge(tvl, borrows_merged, on='vaultName', how='outer').fillna(0)


numeric_columns = ['TVL at Start (USD)', 'TVL at End (USD)', 'TVL at Start (Asset)', 'TVL at End (Asset)', 'Total Borrows at Start (USD)', 'Total Borrows at End (USD)']
for col in numeric_columns:
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')

#Fill NaN values with 0 if needed
final_df = final_df.fillna(0)

# calc total supply
final_df['Borrow at Start (Asset)'] = final_df['Total Borrows at Start (USD)'] / (final_df['TVL at Start (USD)'] / final_df['TVL at Start (Asset)'])
final_df['Borrow at End (Asset)'] = final_df['Total Borrows at End (USD)'] / (final_df['TVL at End (USD)'] / final_df['TVL at End (Asset)'])

final_df['Total Supply at Start (USD)'] = final_df['TVL at Start (USD)'] + final_df['Total Borrows at Start (USD)']
final_df['Total Supply at End (USD)'] = final_df['TVL at End (USD)'] + final_df['Total Borrows at End (USD)']
final_df['Total Supply at Start (Asset)'] = final_df['TVL at Start (Asset)'] + final_df['Borrow at Start (Asset)']
final_df['Total Supply at End (Asset)'] = final_df['TVL at End (Asset)'] + final_df['Borrow at End (Asset)']

# calc changes over epoch
final_df['Change in Supply (USD)'] = final_df['Total Supply at End (USD)'] - final_df['Total Supply at Start (USD)']
final_df['Change in Supply (USD; %)'] = (final_df['Change in Supply (USD)'] / final_df['Total Supply at Start (USD)']) * 100
final_df['Change in Supply (Asset; #)'] = final_df['Total Supply at End (Asset)'] - final_df['Total Supply at Start (Asset)']
final_df['Change in Supply (Asset; %)'] = (final_df['Change in Supply (Asset; #)'] / final_df['Total Supply at Start (Asset)']) * 100

final_df['Change in Borrow (USD, $)'] = final_df['Total Borrows at End (USD)'] - final_df['Total Borrows at Start (USD)']
final_df['Change in Borrows (Asset, #)'] = final_df['Borrow at End (Asset)'] - final_df['Borrow at Start (Asset)']
final_df['Change in Borrows (Percent, USD, %)'] = (final_df['Change in Borrow (USD, $)'] / final_df['Total Borrows at Start (USD)']) * 100
final_df['Change in Borrows (Asset, %)'] = (final_df['Change in Borrows (Asset, #)'] / final_df['Borrow at Start (Asset)']) * 100

# reorder columns
final_df = final_df[[
    'vaultName',
    'Total Supply at Start (USD)',
    'Total Supply at End (USD)',
    'Total Supply at Start (Asset)',
    'Total Supply at End (Asset)',
    'Change in Supply (USD)',
    'Change in Supply (USD; %)',
    'Change in Supply (Asset; #)',
    'Change in Supply (Asset; %)',
    'Total Borrows at Start (USD)',
    'Total Borrows at End (USD)',
    'Borrow at Start (Asset)',
    'Borrow at End (Asset)',
    'Change in Borrow (USD, $)',
    'Change in Borrows (Asset, #)',
    'Change in Borrows (Percent, USD, %)',
    'Change in Borrows (Asset, %)'
]]

# round numeric columns to 2 decimal points
numeric_cols = final_df.select_dtypes(include=['float', 'int']).columns
final_df[numeric_cols] = final_df[numeric_cols].round(2)

# Replace inf, -inf, and NaN with None (might need this!)
final_df = final_df.replace([np.inf, -np.inf, np.nan], None)

print(final_df)


from datetime import date
today = date.today().strftime("%Y-%m-%d")
# final_df.to_csv(f'emissions_analysis_{today}.csv', index=False)

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

# variable to access gspread lib
access = gspread.authorize(creds)


wb_id_dest = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'

sheet_id_dest = 1189222560

wb_dest = access.open_by_key(wb_id_dest)

# define sheet location
sheet_dest = wb_dest.get_worksheet_by_id(sheet_id_dest)

# Convert DataFrame to list of lists, replacing None with empty string
data_to_write = [final_df.columns.tolist()] + [['' if v is None else v for v in row] for row in final_df.values.tolist()]

# write data
sheet_dest.clear()  # Clear existing content
sheet_dest.update(data_to_write, value_input_option='USER_ENTERED')

print(f"Data exported to Tokenomics Model. Last refresh {today}")

         vaultName  Total Supply at Start (USD)  Total Supply at End (USD)  \
0         ionAEROb                 1.597806e+05                  267973.44   
1         ionEURCb                 2.314457e+05                   61689.86   
2         ionLUSDo                 2.163000e+01                      21.63   
3         ionMBTCm                 1.011712e+08                95445972.02   
4        ionMODEmi                 1.348229e+05                  794472.19   
5          ionOGNb                 3.366160e+03                   13680.34   
6           ionOPo                 9.580100e+04                  151863.82   
7          ionRSRb                 6.442905e+05                 1674607.88   
8          ionSNXo                 1.064202e+05                  162466.14   
9        ionSTONEm                 2.865804e+05                  571947.22   
10        ionUSDCb                 2.227363e+06                 1895312.65   
11        ionUSDCm                 1.478792e+06                 

## Emission Analysis Since Start of Incentives

In [11]:
import pandas as pd
import numpy as np
from datetime import date
from google.colab import auth
import gspread
from google.auth import default

def get_closest_date_data(df, target_date):
    """Get data for closest date on or before target_date"""
    mask = df['date'] <= target_date
    if mask.any():
        closest_date = df[mask]['date'].max()
        return df[df['date'] == closest_date].copy()
    return pd.DataFrame()

#  copies of the dataframes
alltime_protocol_tvl = protocol_tvl.copy()
alltime_protocol_tvl_notional = protocol_tvl_notional.copy()
alltime_borrows = borrows.copy()

# Date formatting
alltime_protocol_tvl['date'] = pd.to_datetime(alltime_protocol_tvl['date'], utc=True)
alltime_protocol_tvl_notional['date'] = pd.to_datetime(alltime_protocol_tvl_notional['date'], utc=True)
alltime_borrows['date'] = pd.to_datetime(alltime_borrows['date'], utc=True)

# Rename columns
alltime_protocol_tvl = alltime_protocol_tvl.rename(columns={'ionicVault': 'vaultName'})
alltime_protocol_tvl_notional = alltime_protocol_tvl_notional.rename(columns={'ionicVault': 'vaultName', 'TVL': 'TVL_notional'})

# Define dates
emissions_start = pd.Timestamp('2024-09-15', tz='UTC')
current_date = pd.Timestamp.now(tz='UTC')

# Get TVL data at start date
start_tvl = get_closest_date_data(alltime_protocol_tvl, emissions_start)
tvl_start = start_tvl[['vaultName', 'TVL']].rename(columns={'TVL': 'TVL at Start'})

# Get TVL data at end date
end_tvl = get_closest_date_data(alltime_protocol_tvl, current_date)
tvl_end = end_tvl[['vaultName', 'TVL']].rename(columns={'TVL': 'TVL at End'})

# Get borrow data
def get_latest_borrows(df, date_limit):
    df_filtered = df[df['date'] <= date_limit].copy()
    if not df_filtered.empty:
        return df_filtered.sort_values('date').groupby('vaultName', as_index=False).last()
    return pd.DataFrame()

borrows_start = get_latest_borrows(alltime_borrows, emissions_start)
borrows_end = get_latest_borrows(alltime_borrows, current_date)

borrows_start = borrows_start[['vaultName', 'total_borrowed_in_vault_usd']].rename(
    columns={'total_borrowed_in_vault_usd': 'Total Borrowed at Start'})
borrows_end = borrows_end[['vaultName', 'total_borrowed_in_vault_usd']].rename(
    columns={'total_borrowed_in_vault_usd': 'Total Borrowed at End'})

# Merge data - keeping vaultName throughout
df = pd.merge(tvl_start, tvl_end, on='vaultName', how='outer')
df = pd.merge(df, borrows_start, on='vaultName', how='outer')
df = pd.merge(df, borrows_end, on='vaultName', how='outer')

# Clean data
df = df.replace('<nil>', np.nan)
for col in df.columns:
    if col != 'vaultName':  # Skip vaultName column
        df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.fillna(0)

# Calculate supply and changes
df['Total Supply at Start'] = df['TVL at Start'] + df['Total Borrowed at Start']
df['Total Supply at End'] = df['TVL at End'] + df['Total Borrowed at End']

# Calculate percentage changes
def safe_pct_change(new, old):
    return np.where(old != 0, ((new - old) / old) * 100,
                   np.where(new != 0, np.inf, 0))

df['Change in Supply %'] = safe_pct_change(df['Total Supply at End'], df['Total Supply at Start'])
df['Change in Borrow %'] = safe_pct_change(df['Total Borrowed at End'], df['Total Borrowed at Start'])

# Final dataframe with desired columns
result_df = df[[
    'vaultName',
    'Total Supply at Start',
    'Total Borrowed at Start',
    'Total Supply at End',
    'Total Borrowed at End',
    'Change in Supply %',
    'Change in Borrow %'
]].round(2)

# Replace infinity values with None for Google Sheets
result_df = result_df.replace([np.inf, -np.inf], None)

# Export to Google Sheets
auth.authenticate_user()
creds, _ = default()
access = gspread.authorize(creds)

WORKBOOK_ID = '1tWPMKIqRxg_noABRmQLhti0qXwG3c8bM30bvdzvxruE'
SHEET_ID = '1466607796'

wb = access.open_by_key(WORKBOOK_ID)
sheet = wb.get_worksheet_by_id(int(SHEET_ID))

# Convert to list format for sheets, replacing None with empty string
data_to_write = [result_df.columns.tolist()] + \
                [['' if v is None else v for v in row] for row in result_df.values.tolist()]

# Write to sheet
sheet.clear()
sheet.update(data_to_write, value_input_option='USER_ENTERED')

print(f"Emissions impact analysis exported to Tokenomics Model. Last refresh {date.today().strftime('%Y-%m-%d')}")

Emissions impact analysis exported to Tokenomics Model. Last refresh 2024-12-04
